In [22]:
import pandas as pd
import tensorflow as tf 
import gc
import joblib
from features import extract_sequence, implied_probability, american_to_decimal, fixed_length, split_for_datetime, time_for_75
from tensorflow.keras.models import load_model
import numpy as np

In [23]:
dftp = pd.read_csv("timestampsDatetimeA.csv", header=None, names=range(231))
dftp.head()

C:\Users\maxbr\AppData\Local\Temp\ipykernel_9992\993521357.py:1: DtypeWarning: Columns (206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230) have mixed types. Specify dtype option on import or set low_memory=False.
  dftp = pd.read_csv("timestampsDatetimeA.csv", header=None, names=range(231))


,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,fighter,event,date,opponent,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
1,Mansur Abdul-Malik,UFC,2025-02-23,Nick Klein,2025-02-13 14:17:00 - price: -549,2025-02-13 20:33:00 - price: -549,2025-02-14 18:09:00 - price: -562,2025-02-14 18:13:00 - price: -532,2025-02-14 19:09:00 - price: -569,2025-02-14 19:11:00 - price: -569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Daichi Abe,UFC Fight Night 132: Cowboy vs. Edwards,2018-06-23,Li Jingliang,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,John Adajar,Road to UFC 4,2022-06-10,Han Seul Kim,2022-06-07 10:00:00 - price: -120,2022-06-07 10:18:00 - price: +105,2022-06-07 22:02:00 - price: -105,2022-06-08 13:08:00 - price: +100,2022-06-08 13:26:00 - price: +115,2022-06-08 13:50:00 - price: +118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Shamil Abdurakhimov,UFC 283,2023-01-21,Jailton Almeida,2023-01-06 10:00:00 - price: +600,2023-01-06 13:20:00 - price: +583,2023-01-07 19:48:00 - price: +588,2023-01-08 15:40:00 - price: +585,2023-01-08 23:16:00 - price: +593,2023-01-08 23:54:00 - price: +596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
dftp.columns = dftp.iloc[0]
dftp = dftp[1:]
dftp.reset_index(drop=True, inplace=True)
dftp.head()

,fighter,event,date,opponent,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,Mansur Abdul-Malik,UFC,2025-02-23,Nick Klein,2025-02-13 14:17:00 - price: -549,2025-02-13 20:33:00 - price: -549,2025-02-14 18:09:00 - price: -562,2025-02-14 18:13:00 - price: -532,2025-02-14 19:09:00 - price: -569,2025-02-14 19:11:00 - price: -569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Daichi Abe,UFC Fight Night 132: Cowboy vs. Edwards,2018-06-23,Li Jingliang,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Adajar,Road to UFC 4,2022-06-10,Han Seul Kim,2022-06-07 10:00:00 - price: -120,2022-06-07 10:18:00 - price: +105,2022-06-07 22:02:00 - price: -105,2022-06-08 13:08:00 - price: +100,2022-06-08 13:26:00 - price: +115,2022-06-08 13:50:00 - price: +118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shamil Abdurakhimov,UFC 283,2023-01-21,Jailton Almeida,2023-01-06 10:00:00 - price: +600,2023-01-06 13:20:00 - price: +583,2023-01-07 19:48:00 - price: +588,2023-01-08 15:40:00 - price: +585,2023-01-08 23:16:00 - price: +593,2023-01-08 23:54:00 - price: +596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hamdy Abdelwahab,UFC,2025-02-01,Jamal Pogues,2025-01-14 07:53:00 - price: +105,2025-01-14 12:39:00 - price: +113,2025-01-15 08:41:00 - price: +110,2025-01-16 12:13:00 - price: +111,2025-01-16 14:27:00 - price: +111,2025-01-16 15:57:00 - price: +112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df = pd.read_csv('simData48Clean.csv')
df.head()

,date,event,fighter1,fighter1 - open,fighter1 - 48h,fighter1 - close,fighter2,fighter2 - open,fighter2 - 48h,fighter2 - close,winner
0,2025-03-09,UFC,Curtis Blaydes,2025-01-18 02:57:00 - price: -275,2025-03-06 15:01:00 - price: -339,2025-03-08 15:01:00 - price: -335,Rizvan Kuniev,2025-01-18 02:57:00 - price: +210,2025-03-06 15:01:00 - price: +259,2025-03-08 15:01:00 - price: +261,NaN
1,2025-03-09,UFC,Brunno Ferreira,2025-02-24 18:07:00 - price: +105,2025-03-06 21:01:00 - price: +116,2025-03-08 21:01:00 - price: +112,Armen Petrosyan,2025-02-24 18:07:00 - price: -125,2025-03-06 21:01:00 - price: -141,2025-03-08 21:01:00 - price: -134,Brunno Ferreira
2,2025-03-09,UFC,Rafael Fiziev,2025-02-27 05:11:00 - price: -139,2025-03-07 01:01:00 - price: -163,2025-03-08 23:01:00 - price: -159,Justin Gaethje,2025-02-27 05:11:00 - price: +112,2025-03-07 01:01:00 - price: +133,2025-03-08 23:01:00 - price: +131,Justin Gaethje
3,2025-03-09,UFC,Francis Marshall,2025-02-24 18:15:00 - price: +290,2025-03-06 17:01:00 - price: +247,2025-03-08 19:01:00 - price: +197,Mairon Santos,2025-02-24 18:15:00 - price: -370,2025-03-06 17:01:00 - price: -322,2025-03-08 19:01:00 - price: -245,Mairon Santos
4,2025-03-09,UFC,John Castaneda,2025-03-03 09:01:00 - price: -112,2025-03-06 15:01:00 - price: -106,2025-03-08 15:01:00 - price: -108,Chris Gutierrez,2025-03-03 09:01:00 - price: -112,2025-03-06 15:01:00 - price: -116,2025-03-08 15:01:00 - price: -112,NaN


In [26]:
print(df.isna().any(axis=1).sum())
df = df.dropna().reset_index(drop=True)
print(df.isna().any(axis=1).sum())

285
0


In [27]:
def calculate_profit(wager, price):
    if price > 0:
        profit = (price/100)*wager
    else:
        profit = (100/abs(price))*wager
    return profit

In [ ]:
import random

def perform_sim(wager, model, size):
    bankroll = wager * len(df)

    profit_dict = {
    'favorite_at_open': bankroll,
    'favorite_at_close': bankroll,
    'favorite_at_48h': bankroll,

    'favorite_ML': bankroll,

    'underdog_at_open': bankroll,
    'underdog_at_close': bankroll,
    'underdog_at_48h': bankroll,

    'underdog_ML': bankroll,

    'favorite_random': bankroll,
    'underdog_random': bankroll,
    
    'true_random': bankroll
    }   

    for index, row in df.iterrows():
        print(index)
        fighter1 = row['fighter1']
        fighter2 = row['fighter2']
        winner = row['winner']

        model = load_model(model)


        #get open line
        open1 = int(row['fighter1 - open'].split('price: ')[1])
        open2 = int(row['fighter2 - open'].split('price: ')[1])

        close1 = int(row['fighter1 - close'].split('price: ')[1])
        close2 = int(row['fighter2 - close'].split('price: ')[1])

        forty81 = int(row['fighter1 - 48h'].split('price: ')[1])
        forty82 = int(row['fighter2 - 48h'].split('price: ')[1])

        lines = {
            fighter1: [open1, forty81, close1],
            fighter2: [open2, forty82, close2]
        }

        fighter1_avg = sum(lines[fighter1])/len(lines[fighter1])
        fighter2_avg = sum(lines[fighter2])/len(lines[fighter2])

        #get indice for fav and dog
        if fighter1_avg < fighter2_avg:
            favorite_mean = 1
            dog_mean = 2
        else:
            favorite_mean = 2
            dog_mean = 1
            
        #find the sequence
        fav_frame = dftp[(dftp['fighter'] == row[f'fighter{favorite_mean}']) 
                    & (dftp['opponent'] == row[f'fighter{dog_mean}'])
                    & (dftp['date'] == row['date'])]
        
        dog_frame = dftp[(dftp['fighter'] == row[f'fighter{dog_mean}']) 
                    & (dftp['opponent'] == row[f'fighter{favorite_mean}'])
                    & (dftp['date'] == row['date'])]
        
        #extract sequences
        fav_sequence = extract_sequence(fav_frame)
        dog_sequence = extract_sequence(dog_frame)

        #generate input data
        fav_seq = fixed_length(fav_sequence, size, 'linear')
        dog_seq = fixed_length(dog_sequence, size, 'linear')

        fav_action = model.predict(np.array(fav_seq, dtype=np.float32).reshape(1, 20, 1))[0]
        dog_action = model.predict(np.array(dog_seq, dtype=np.float32).reshape(1, 20, 1))[0]

        #Test changed to decimal
        if(fav_action >= 0.5):
            fav_ML = lines[row[f'fighter{favorite_mean}']][2]
        else:
            fav_ML = lines[row[f'fighter{favorite_mean}']][1]

        if(dog_action >= 0.5):
            dog_ML = lines[row[f'fighter{dog_mean}']][2]
        else:
            dog_ML = lines[row[f'fighter{dog_mean}']][1]

        line_list = lines[winner]

        if fav_ML in line_list:
            profit_dict['favorite_ML']+=calculate_profit(wager, fav_ML)
        else:
            profit_dict['favorite_ML']-=wager
        
        if dog_ML in line_list:
            profit_dict['underdog_ML']+=calculate_profit(wager, dog_ML)
        else:
            profit_dict['underdog_ML']-=wager
            

        for_random = [open1, open2, forty81, close1, forty82, close2]

        neg_lines = []
        pos_lines = []
        for line in for_random:
            if(line > 0):
                pos_lines.append(line)
            else:
                neg_lines.append(line)

        if(len(pos_lines) >0):
            #dog random blindly takes a line
            random_dog = random.choice(pos_lines)
        else:
            #no pos line default to closest to pos
            random_dog = max(for_random)

        if(len(neg_lines)> 0):
            #fav random blindly takes a line
            random_fav = random.choice(neg_lines)
        else:
            random_fav = min(for_random)


        #if random fav picked a winning line
        if random_fav in line_list:
            profit_dict['favorite_random']+=calculate_profit(wager, random_fav)
        else:
            profit_dict['favorite_random']-=wager

        #if random dog picked a winning line
        if random_dog in line_list:
            profit_dict['underdog_random']+=calculate_profit(wager, random_dog)
        else:
            profit_dict['underdog_random']-=wager



        #true random
        random_fighter = row[f'fighter{random.randint(1,2)}']
        random_line = random.randint(0,2)
        true_random = lines[random_fighter][random_line]
        

        #if true random picked a winning line
        if true_random in line_list:
            profit_dict['true_random']+=calculate_profit(wager, true_random)
        else:
            profit_dict['true_random']-=wager

        #open fav line wins
        if(line_list[0] < 0):
            #favorite open bettors win
            profit_dict['favorite_at_open']+=calculate_profit(wager, line_list[0])
            #dog open bettor loses
            profit_dict['underdog_at_open']-=wager
        #open fav line loses
        else:
            profit_dict['favorite_at_open']-=wager
            profit_dict['underdog_at_open']+=calculate_profit(wager, line_list[0])

        #48h fav wins
        if(line_list[1] < 0):
            #favorite 48h bettors win
            profit_dict['favorite_at_48h']+=calculate_profit(wager, line_list[1])
            #dog 48h bettor loses
            profit_dict['underdog_at_48h']-=wager
        #48h fav line loses
        else:
            profit_dict['favorite_at_48h']-=wager
            profit_dict['underdog_at_48h']+=calculate_profit(wager, line_list[1])


        if(line_list[2] < 0):
            #favorite open bettors win
            profit_dict['favorite_at_close']+=calculate_profit(wager, line_list[2])
            #dog open bettor loses
            profit_dict['underdog_at_close']-=wager
        #open fav line loses
        else:
            profit_dict['favorite_at_close']-=wager
            profit_dict['underdog_at_close']+=calculate_profit(wager, line_list[2])


    for k in profit_dict:
        profit_dict[k] = (profit_dict[k] - bankroll) / bankroll

    return(profit_dict)



In [ ]:
simpleRNN = 'simpleRNN.keras'

In [30]:
simpleRNNSim = perform_sim(100, simpleRNN, 20)

0
1/1 [==============================] - ETA: 0s

c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()
c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()


1/1 [==============================] - 0s 31ms/step
1
1/1 [==============================] - 0s 86ms/step


c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()
c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()


1/1 [==============================] - 0s 31ms/step
2
1/1 [==============================] - 0s 16ms/step


...

532
1/1 [==============================] - 0s 31ms/step


c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()
c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()


533


c:\Users\maxbr\OneDrive\Desktop\CLVSim\CLVSim\CLV Sim\Sim\features.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame = frame.applymap(lambda x: x.split(' - price: ')[1].strip()


KeyboardInterrupt: 